In [133]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from lightfm import LightFM
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from features2 import change as changedf #импорт фичей

In [165]:
data = pd.read_csv("train_1.csv", sep=';')
add_data = pd.read_csv("train_2.csv", sep = ';')

C:\Users\Egor\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Egor\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [167]:
changedf(add_data)

In [136]:
add_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               200000 non-null  int64  
 1   MM_IN_BANK       200000 non-null  int64  
 2   MM_W_CARD        200000 non-null  int64  
 3   AGE              200000 non-null  int64  
 4   GENDER           200000 non-null  int64  
 5   EDUCATION_LEVEL  200000 non-null  int64  
 6   MARITAL_STATUS   200000 non-null  int64  
 7   DEPENDANT_CNT    200000 non-null  int64  
 8   INCOME_MAIN_AMT  200000 non-null  float64
 9   REG_CODE         200000 non-null  object 
 10  REG_ZP           200000 non-null  int32  
 11  NEW_INC          200000 non-null  float64
 12  srinc            200000 non-null  int32  
dtypes: float64(2), int32(2), int64(8), object(1)
memory usage: 18.3+ MB


In [168]:
data_train = data.sample(frac = 0.1)

add_data = add_data.drop(columns = 'INCOME_MAIN_AMT')
add_data = add_data.drop(columns = 'REG_CODE')
data_train = data_train.drop(columns = ['LOCATION_NAME'])

In [ ]:
data_train.head(3)

,ID,PROD_TYPE,TRANS_DTTM,MCC_CODE,SUM_TRANS
3840461,500000000000273954,1,21.07.2021 13:05:25,5691,9182
2706798,500000000006039955,1,04.09.2021 20:17:28,5411,"471,44"
7618000,500000000052556625,2,31.08.2021 15:15:22,5912,"459,9"


In [161]:
def additional_data_transform(add_data):
    features = list(add_data.columns)
    features.remove('ID')
    bins = 10
    for i in features:
        add_data[i] = pd.cut(add_data[i], bins)  
    add_data_encode = pd.get_dummies(add_data, columns=features)    
    return add_data_encode    
def model_training(sparse_user_item, sample_weight, user_features):
    model = LightFM(loss = 'warp', 
                    learning_rate = 0.00001, 
                    user_alpha = 0.01, 
                    random_state = 30,
                    k = 5,
                    n = 10,
                    max_sampled=100)
    model.fit(sparse_user_item,
              sample_weight=coo_matrix(sample_weight),
              user_features=csr_matrix(user_features.values).tocsr(),
              epochs=20, 
              num_threads=20,
              verbose=True,)
    return model

def estimate(data, add_data):
    user_item_mat = pd.pivot_table(data, index='ID', columns='MCC_CODE', aggfunc='count', fill_value=0)
    user_item_mat = user_item_mat.astype(float) 
    user_item_mat = user_item_mat.droplevel(0, axis = 1)
    user_item_mat = user_item_mat.groupby(user_item_mat.columns, axis = 1).sum()
    
    mms = MinMaxScaler()
    user_item_mat = pd.DataFrame(np.array(mms.fit_transform(user_item_mat)), index=user_item_mat.index, columns=user_item_mat.columns)
    sparse_user_item = csr_matrix(user_item_mat).tocsr()
     
    model = model_training(sparse_user_item, user_item_mat, add_data)
    train_precision = precision_at_k(model, sparse_user_item, user_features=csr_matrix(add_data.values).tocsr(), k=3).mean()
    print(f"Mean train precision by 3 k {train_precision}")
    
    userids = user_item_mat.index.values
    itemids = user_item_mat.columns.values

    mat_userids = np.arange(len(userids))
    mat_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(mat_itemids, itemids))
    id_to_userid = dict(zip(mat_userids, userids))

    item_to_id = dict(zip(itemids, mat_itemids))
    user_to_id = dict(zip(userids, mat_userids))

    return model, item_to_id, user_to_id

def predict(data, add_data, model, item_to_id, user_to_id):
    users_ids_row = data['ID'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data['MCC_CODE'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    predictions = model.predict(user_ids=users_ids_row, item_ids=items_ids_row, user_features=csr_matrix(add_data.values).tocsr())
    data['score'] = predictions
    predict_result = data[['ID','MCC_CODE','score']].drop_duplicates().sort_values(by=['ID','score'], ascending=False).groupby('ID')['MCC_CODE'].unique().reset_index()
    predict_result = predict_result.set_index('ID')
    return predict_result['MCC_CODE'].apply(lambda x: x[:3])

In [170]:
add_data = additional_data_transform(add_data)

In [171]:
model, item_id, user_id = estimate(data_train, add_data)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [01:24<00:00,  4.23s/it]


Mean train precision by 3 k 0.07578647881746292


In [162]:
print(predict(data_train, add_data, model, item_id, user_id))

ID
500000000000003702    [5411, 5691, 4814]
500000000000004100    [5812, 5533, 5912]
500000000000004523    [5921, 5411, 5651]
500000000000004529    [5411, 4131, 4121]
500000000000004718    [5912, 5921, 5411]
                             ...        
500000004694872697                [4112]
500000004694884558    [5411, 5311, 5661]
500000004694909288                [5977]
500000004694952000    [5814, 5993, 5499]
500000004694957908    [5411, 4111, 4900]
Name: MCC_CODE, Length: 186785, dtype: object


In [144]:
add_data.head(5)


,ID,"MM_IN_BANK_(-0.332, 33.2]","MM_IN_BANK_(33.2, 66.4]","MM_IN_BANK_(66.4, 99.6]","MM_IN_BANK_(99.6, 132.8]","MM_IN_BANK_(132.8, 166.0]","MM_IN_BANK_(166.0, 199.2]","MM_IN_BANK_(199.2, 232.4]","MM_IN_BANK_(232.4, 265.6]","MM_IN_BANK_(265.6, 298.8]",...,"srinc_(-0.001, 0.1]","srinc_(0.1, 0.2]","srinc_(0.2, 0.3]","srinc_(0.3, 0.4]","srinc_(0.4, 0.5]","srinc_(0.5, 0.6]","srinc_(0.6, 0.7]","srinc_(0.7, 0.8]","srinc_(0.8, 0.9]","srinc_(0.9, 1.0]"
0,500000000000003702,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,500000000000004100,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,500000000000004523,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,500000000000004529,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,500000000000004718,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [169]:
add_data.head(5)

,ID,MM_IN_BANK,MM_W_CARD,AGE,GENDER,EDUCATION_LEVEL,MARITAL_STATUS,DEPENDANT_CNT,REG_ZP,NEW_INC,srinc
0,500000000000003702,5,5,35,2,3,3,0,35962,87000.0,1
1,500000000000004100,108,0,34,2,3,3,2,51344,60000.0,1
2,500000000000004523,0,0,35,1,1,3,0,37828,61000.0,1
3,500000000000004529,6,6,38,1,5,3,0,41958,318000.0,1
4,500000000000004718,18,5,39,1,-1,3,0,39349,34000.0,0
